In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from inputs.constants import DATA_PATH
pd.options.plotting.backend = 'plotly'

In [ ]:
def get_base_vacc_data():
    vacc_df = pd.read_csv(DATA_PATH / 'aus_vax_data.csv', index_col=424)
    vacc_df.index = pd.to_datetime(vacc_df.index, infer_datetime_format=True)
    return vacc_df.sort_index()

def get_vacc_data_masks(df):
    masks = {
        'age_16_2dose': [
            c for c in df.columns if 'Number of people fully vaccinated' in c and 
            not any(s in c for s in [' - M - ', ' - F - '])
        ],
        'age_16_3dose': [
            col for col in df.columns if 
            'National' in col and 
            any([s in col for s in ['who have received 3 doses', 'with 3 or more doses', 'with more than two doses', 'with more than 2 doses']]) and 
            any([s in col for s in ['16', '18']]) and
            not any([s in col for s in ['increase', 'Percentage', 'Indigenous']])
        ],
        'age_16_4dose': [
            col for col in df.columns if
            'National' in col and
            any([s in col for s in ['Winter Boosters number', 'who have received 4 doses', 'Fourth dose number']]) and
            not any([s in col for s in ['30', '65', 'increase']])
        ],
        'age_12_15_2dose': [
            col for col in df.columns if
            '12-15' in col and
            any([s in col for s in ['National', 'Age group']]) and
            any([s in col for s in ['2 doses', 'fully vaccinated']]) and
            not any([s in col for s in ['Indigenous', 'Population', '- F -', '- M -']])
        ],
        'age_5_11_2dose': [
            col for col in df.columns if
            'National' in col and
            '5-11' in col and
            any([s in col for s in ['2 doses', 'fully vaccinated']])
        ],
    }
    masks['age_16_2dose'].sort()
    return masks

def plot_full_vacc(full_vacc_masks, df):
    fig = go.Figure()
    for a, age in enumerate(full_vacc_masks):
        prop = int(np.round(a / len(masks['age_16_2dose']) * 250.0))
        colour = f'rgb({prop},{250 - prop},250)'
        trace_name = age.replace('- Number of people fully vaccinated', '').replace('Age group - ', '')
        data = df[age].dropna()
        fig.add_trace(go.Scatter(x=data.index, y=data, name=trace_name, line={'color': colour}))
    return fig

def plot_program_coverage(program_masks, df):
    fig = make_subplots(rows=2, cols=2, subplot_titles=program_masks)
    for m in range(len(program_masks)):
        col = m % 2 + 1
        row = int(np.floor(m / 2)) + 1
        fig.add_traces(px.line(df[masks[program_masks[m]]]).data, rows=row, cols=col)
    fig.update_layout(height=600, showlegend=False, title='Coverage by program')
    return fig

def add_booster_data_to_vacc(df, rolling_window, immune_duration):
    df['adult booster'] = df.loc[:, masks['age_16_3dose'] + masks['age_16_4dose']].sum(axis=1)
    df = vacc_df.drop(datetime(2022, 7, 8))
    df['adult booster smooth'] = df.loc[:, 'adult booster'].rolling(rolling_window).mean()
    df['incremental adult booster'] = df['adult booster smooth'].diff()
    df['boosted in preceding'] = df['incremental adult booster'].rolling(immune_duration).sum()
    df['prop boosted in preceding'] = df['boosted in preceding'] / df['National - Population 16 and over']
    return df

In [ ]:
vacc_df = get_base_vacc_data()
masks = get_vacc_data_masks(vacc_df)
vacc_df = add_booster_data_to_vacc(vacc_df, 7, 100)

In [ ]:
plot_full_vacc(masks['age_16_2dose'], vacc_df)

In [ ]:
plot_program_coverage(['age_16_3dose', 'age_16_4dose', 'age_12_15_2dose', 'age_5_11_2dose'], vacc_df)

In [ ]:
vacc_df[['adult booster', 'adult booster smooth']].plot(title='Cumulative boosters in ages 16+')

In [ ]:
vacc_df['incremental adult booster'].plot(title='Smoothed incremental additional doses by day')

In [ ]:
vacc_df['boosted in preceding'].plot(title=f'Vaccinated in preceding period')

In [ ]:
vacc_df['prop boosted in preceding'].plot(title=f'Proportion vaccinated in preceding period')